In [ ]:
import pandas as pd
import requests
import time


In [ ]:

# Load your data
df = pd.read_csv('test.csv')

# Load NOAA API token from file
with open('API_token.txt', 'r') as token_file:
    NOAA_TOKEN = token_file.read().strip()

print("NOAA Token loaded successfully: " + NOAA_TOKEN)




In [ ]:
BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
HEADERS = {"token": NOAA_TOKEN}

In [ ]:
import pandas as pd
import requests
import time

# Load your data
df = pd.read_csv('test.csv')

# Load NOAA API token from file
with open('API_token.txt', 'r') as token_file:
    NOAA_TOKEN = token_file.read().strip()

print("NOAA Token loaded successfully: " + NOAA_TOKEN)

BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
HEADERS = {"token": NOAA_TOKEN}

def parse_response_for_errors(response):
    """
    Parses the response object for errors and prints detailed feedback.
    """
    if response.status_code != 200:
        print("Error detected in response:")
        print(f"Status Code: {response.status_code}")
        print(f"URL: {response.url}")
        print(f"Response Text: {response.text}")
        try:
            error_details = response.json()
            print("Error Details:", error_details)
        except ValueError:
            print("Response is not in JSON format.")
        return True  # Indicates an error occurred
    return False  # No errors detected

def make_request(url, params, retries=5):
    """
    Makes a request to the given URL with retries for handling 503 errors.
    """
    for attempt in range(retries):
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code == 503:
            print(f"503 Service Unavailable. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)  # Exponential backoff
            continue
        return response
    print("Failed after retries. Server is unavailable.")
    return None

def fetch_weather_data(lat, lon, start_date, end_date, datasetid="GHCND", datatypeid="TAVG", limit=1000, retries=5):
    """
    Fetches weather data from NOAA API for the given parameters.
    """
    # Find nearest station first
    stations_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
    params = {
        "datasetid": datasetid,
        "limit": 1,
        "latitude": lat,
        "longitude": lon,
    }
    response = make_request(stations_url, params, retries=retries)
    if not response or parse_response_for_errors(response):
        return None

    stations = response.json().get('results')
    if not stations:
        print("No stations found near:", lat, lon)
        return None

    station_id = stations[0]['id']

    # Now get the weather data
    data_params = {
        "datasetid": datasetid,
        "datatypeid": datatypeid,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": limit,
        "units": "metric"
    }
    response = make_request(BASE_URL, data_params, retries=retries)
    if not response or parse_response_for_errors(response):
        return None

    return response.json().get("results", [])


In [ ]:
# Sample query to test the function
sample_lat = 43.0731
sample_lon = -89.4012 #madison, WI
sample_start_date = "2023-01-01"
sample_end_date = "2023-01-31"

# Fetch weather data for the sample query
sample_weather_data = fetch_weather_data(
    lat=sample_lat,
    lon=sample_lon,
    start_date=sample_start_date,
    end_date=sample_end_date
)

# Print the sample weather data
print("Sample Weather Data:", sample_weather_data)

In [ ]:
# Example: process the DataFrame row by row and add results
weather_data = []

for _, row in df.iterrows():
    result = fetch_weather_data(
        lat=row["latitude"],
        lon=row["longitude"],
        start_date=row["start_date"],
        end_date=row["end_date"]
    )
    weather_data.append(result)
    time.sleep(1)  # Avoid rate limiting

df["weather"] = weather_data

# Save to new file
df.to_csv("weather_data_filled.csv", index=False)